# Specifications
Your stakeholder only wants you to include information for movies based on the following specifications:

- Exclude any movie with missing values for genre or runtime
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre)
- Include only movies that were released 2000 - 2021 (include 2000 and 2021)
- Include only movies that were released in the United States

In [1]:
# Imports

import pandas as pd
import numpy as np

In [2]:
basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [3]:
ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t', low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1968
1,tt0000002,5.8,263
2,tt0000003,6.5,1814
3,tt0000004,5.6,178
4,tt0000005,6.2,2611


In [4]:
akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep='\t', low_memory=False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


# Data Cleaning and Filtering

## Akas Dataframe

In [5]:
akas.replace({'\\N':np.nan}, inplace=True)
print(akas.info())
akas.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35831905 entries, 0 to 35831904
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.1+ GB
None


titleId                   0
ordering                  0
title                     5
region              1889447
language            6595771
types              30283173
attributes         35572943
isOriginalTitle        2109
dtype: int64

In [6]:
# Keep only US movies

akas = akas[akas['region']=='US']
akas['region'].value_counts()

US    1435233
Name: region, dtype: int64

## Basics Dataframe

In [7]:
# Null values are imputed as "\N", therefore I will correct this using pandas
basics.replace({'\\N':np.nan}, inplace=True)
print(basics.info()) 
print('\n')
basics.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9835151 entries, 0 to 9835150
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 675.3+ MB
None




tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1330643
endYear           9728816
runtimeMinutes    6935414
genres             443035
dtype: int64

In [8]:
# Exclude any movie with missing values for genre or runtime
basics.dropna(subset=['genres', 'runtimeMinutes'], inplace = True)
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear          165212
endYear           2773304
runtimeMinutes          0
genres                  0
dtype: int64

In [9]:
# Keep only titleType==Movie
basics['titleType'].value_counts()

tvEpisode       1433895
short            600493
movie            382010
video            180360
tvMovie           91525
tvSeries          90376
tvSpecial         18125
tvMiniSeries      17165
tvShort            8808
videoGame           323
Name: titleType, dtype: int64

In [10]:
basics = basics[basics['titleType'] == 'movie']
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382010 entries, 8 to 9835101
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          382010 non-null  object
 1   titleType       382010 non-null  object
 2   primaryTitle    382010 non-null  object
 3   originalTitle   382010 non-null  object
 4   isAdult         382010 non-null  object
 5   startYear       375572 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  382010 non-null  object
 8   genres          382010 non-null  object
dtypes: object(9)
memory usage: 29.1+ MB


In [11]:
# keep startYear 2000-2022
basics['startYear'] = basics['startYear'].astype(float)
basics['startYear'].dtype

dtype('float64')

In [12]:
basics = basics[(basics['startYear']>=2000)&(basics['startYear']<2022)]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021.0,NaN,94,Documentary
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [13]:
# Eliminate movies that include "Documentary" in genre

documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~documentary]

In [14]:
basics['genres'].str.contains('documentary',case=False).sum()

0

In [15]:
# Keep only US movies

basics = basics[basics['tconst'].isin(akas['titleId'])]

## Ratings Dataframe

In [16]:
ratings.replace({'\\N':np.nan}, inplace=True)
print(ratings.info())
ratings.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308293 entries, 0 to 1308292
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1308293 non-null  object 
 1   averageRating  1308293 non-null  float64
 2   numVotes       1308293 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 29.9+ MB
None


tconst           0
averageRating    0
numVotes         0
dtype: int64

In [17]:
ratings = ratings[ratings['tconst'].isin(akas['titleId'])]

## Final dataframe info

In [18]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1435233 entries, 5 to 35831649
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1435233 non-null  object
 1   ordering         1435233 non-null  int64 
 2   title            1435233 non-null  object
 3   region           1435233 non-null  object
 4   language         3907 non-null     object
 5   types            978610 non-null   object
 6   attributes       46532 non-null    object
 7   isOriginalTitle  1433888 non-null  object
dtypes: int64(1), object(7)
memory usage: 98.5+ MB


In [19]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81698 entries, 34803 to 9834917
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81698 non-null  object 
 1   titleType       81698 non-null  object 
 2   primaryTitle    81698 non-null  object 
 3   originalTitle   81698 non-null  object 
 4   isAdult         81698 non-null  object 
 5   startYear       81698 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  81698 non-null  object 
 8   genres          81698 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.2+ MB


In [20]:

ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497708 entries, 0 to 1308268
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         497708 non-null  object 
 1   averageRating  497708 non-null  float64
 2   numVotes       497708 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.2+ MB


In [21]:
# Making folder with os
import os
os.makedirs('Data/',exist_ok=True) 

# Confirm folder created
os.listdir("Data/")

['.ipynb_checkpoints',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json']

In [22]:
## Saving dataframes to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)


In [23]:
# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [24]:
# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [25]:
# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1968
1,tt0000002,5.8,263
2,tt0000005,6.2,2611
3,tt0000006,5.1,181
4,tt0000007,5.4,817
